In [53]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score

# ------------------ Config ------------------ #
model_dir = r"C:\Users\azorb\PycharmProjects\Predicting the Yield Curve\Model Fit\Output"
model_folders = ["RW", "LSTM", "AR_YW", "VAR_YW", "DNS_VAR_YW", "DNS_KF_CORR", "DNS_KF"]
horizons = [1, 5, 21, 63, 252]

# ------------------ Metrics ------------------ #
def compute_directional_metrics(y_true, y_pred):
    true_diff = np.sign(np.diff(y_true, axis=0))
    pred_diff = np.sign(np.diff(y_pred, axis=0))
    true_up = true_diff > 0
    pred_up = pred_diff > 0
    match = true_up == pred_up
    acc = match.mean()
    f1 = f1_score(true_up.flatten(), pred_up.flatten(), zero_division=0)
    return acc, f1

results_by_horizon = {h: [] for h in horizons}

# ------------------ Processing ------------------ #
for model in model_folders:
    for h in horizons:
        file_path = os.path.join(model_dir, model, f"{model.lower()}_h{h}_fixed_window_results.csv")
        if not os.path.exists(file_path):
            continue
        df = pd.read_csv(file_path)
        print(file_path)
        y_true = np.array(df["true_yields"].apply(eval).tolist())
        y_pred = np.array(df["forecast_yields"].apply(eval).tolist())

        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
        r2 = r2_score(y_true, y_pred)
        dir_acc, dir_f1 = compute_directional_metrics(y_true, y_pred)

        results_by_horizon[h].append({
            "Model": model,
            "MAE": mae,
            "RMSE": rmse,
            "R2": r2,
            "Directional Accuracy": dir_acc,
            "Directional F1": dir_f1
        })

# ------------------ Display ------------------ #
tables = {}
for h in horizons:
    df = pd.DataFrame(results_by_horizon[h])
    df = df[["Model", "MAE", "RMSE", "R2", "Directional Accuracy", "Directional F1"]]
    tables[h] = df
    df.head(100)

C:\Users\azorb\PycharmProjects\Predicting the Yield Curve\Model Fit\Output\RW\rw_h1_fixed_window_results.csv
C:\Users\azorb\PycharmProjects\Predicting the Yield Curve\Model Fit\Output\RW\rw_h5_fixed_window_results.csv
C:\Users\azorb\PycharmProjects\Predicting the Yield Curve\Model Fit\Output\RW\rw_h21_fixed_window_results.csv
C:\Users\azorb\PycharmProjects\Predicting the Yield Curve\Model Fit\Output\RW\rw_h63_fixed_window_results.csv
C:\Users\azorb\PycharmProjects\Predicting the Yield Curve\Model Fit\Output\RW\rw_h252_fixed_window_results.csv
C:\Users\azorb\PycharmProjects\Predicting the Yield Curve\Model Fit\Output\LSTM\lstm_h1_fixed_window_results.csv
C:\Users\azorb\PycharmProjects\Predicting the Yield Curve\Model Fit\Output\LSTM\lstm_h5_fixed_window_results.csv
C:\Users\azorb\PycharmProjects\Predicting the Yield Curve\Model Fit\Output\LSTM\lstm_h21_fixed_window_results.csv
C:\Users\azorb\PycharmProjects\Predicting the Yield Curve\Model Fit\Output\LSTM\lstm_h63_fixed_window_results.c

In [78]:
tables[252].round(3)

,Model,MAE,RMSE,R2,Directional Accuracy,Directional F1
0,RW,3.459,3.738,-24.672,0.518,0.392
1,LSTM,2.405,3.194,-13.298,0.507,0.478
2,AR_YW,3.062,3.306,-18.403,0.475,0.417
3,VAR_YW,3.074,3.298,-19.145,0.495,0.447
4,DNS_VAR_YW,3.130,3.361,-20.046,0.492,0.440
5,DNS_KF_CORR,2.844,3.979,-38.260,0.483,0.450
6,DNS_KF,2.436,3.215,-24.184,0.492,0.479


In [12]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score, f1_score

# ------------------ Config ------------------ #
model = "RW"
horizons = [1, 5, 21, 63, 252]
base_path = r"C:\Users\azorb\PycharmProjects\Predicting the Yield Curve\Model Fit\Output\RW"

def compute_directional_metrics(y_true, y_pred):
    true_diff = np.sign(np.diff(y_true, axis=0))
    pred_diff = np.sign(np.diff(y_pred, axis=0))
    true_up = true_diff > 0
    pred_up = pred_diff > 0
    match = true_up == pred_up
    acc = match.mean()
    f1 = f1_score(true_up.flatten(), pred_up.flatten(), zero_division=0)
    return acc, f1

results_by_horizon = {}

# ------------------ Processing ------------------ #
for h in horizons:
    file_path = os.path.join(base_path, f"rw_h{h}_fixed_window_results.csv")
    df = pd.read_csv(file_path)
    y_true = np.array(df["true_yields"].apply(eval).tolist())
    y_pred = np.array(df["forecast_yields"].apply(eval).tolist())

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    r2 = r2_score(y_true, y_pred)
    dir_acc, dir_f1 = compute_directional_metrics(y_true, y_pred)

    results_by_horizon[h] = {
        "Model": model,
        "MAE": mae,
        "RMSE": rmse,
        "R2": r2,
        "Directional Accuracy": dir_acc,
        "Directional F1": dir_f1
    }

# ------------------ Display ------------------ #
results_df = pd.DataFrame.from_dict(results_by_horizon, orient='index')
results_df

,Model,MAE,RMSE,R2,Directional Accuracy,Directional F1
1,RW,0.042727,0.063747,0.988464,0.497130,0.429930
5,RW,0.042727,0.063747,0.988464,0.497130,0.429930
21,RW,0.042727,0.063747,0.988464,0.497130,0.429930
63,RW,0.042727,0.063747,0.988464,0.498013,0.430361
252,RW,0.042727,0.063747,0.988464,0.497130,0.429930
